# Jacaranda street trees of LA
> This notebook reads and analyzes the locations of [jacaranda trees](https://en.wikipedia.org/wiki/Jacaranda_mimosifolia#:~:text=Jacaranda%20mimosifolia%20is%20a%20sub,poui%2C%20Nupur%20or%20fern%20tree.) along Los Angeles County streets and uploads them for use with Mapbox GL JS. 

---

#### Load Python tools and Jupyter config

In [1]:
import os
import json
import boto3
import mapbox
import requests
import mercantile
import pandas as pd
import jupyter_black
from io import BytesIO
import geopandas as gpd
import mapbox_vector_tile
from mapbox import Uploader

In [2]:
jupyter_black.load()
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
mapbox_key = os.environ.get("PERSONAL_MAPBOX_TILESET_ACCESS_TOKEN")

---

## Read

#### Read GeoJSON stored on S3

In [3]:
# Extracted from scripts/fetch_la_treekeeper.py
la_url = "https://stilesdata.com/trees/los-angeles/la_street_trees_latlon.geojson"
la_src = gpd.read_file(la_url).to_crs("EPSG:3857")

In [4]:
la_src["place"] = "Los Angeles City"
la_src["jacaranda"] = la_src["tree_common"].str.contains("Jacaranda", case=False)

In [5]:
# We're replacing the legacy LA city trees with fresh data from treekeeper so query them out now
src = (
    gpd.read_file("../data/processed/la_county_tree_locations.geojson")
    .to_crs("EPSG:3857")
    .query('place != "Los Angeles City"')
)

In [6]:
gdf = gpd.GeoDataFrame(pd.concat([src, la_src]).reset_index(drop=True)).drop(
    ["site_id", "tree_common"], axis=1
)

In [7]:
gdf.head()

,id,place,species,ispalm,mexfanpalm,jacaranda,pine,oak,magnolia,category,camphor,ash,crepemyrtle,geometry
0,0.0,Santa Clarita,Afghan Pine,False,False,False,True,False,False,pine,0,0,0,POINT (-13195508.245 4088443.700)
1,1.0,Long Beach,Vacant,False,False,False,False,False,False,other,0,0,0,POINT (-13155502.077 4006002.591)
2,2.0,Pomona,Vacant,False,False,False,False,False,False,other,0,0,0,POINT (-13106569.831 4033498.099)
3,3.0,Pomona,Vacant,False,False,False,False,False,False,other,0,0,0,POINT (-13106681.760 4033533.929)
4,4.0,Pomona,Vacant,False,False,False,False,False,False,other,0,0,0,POINT (-13106698.142 4033534.597)


#### Just the jacarandas

In [8]:
jac_gdf = gdf.query("jacaranda == True").reset_index(drop=True).copy()

#### How many trees?

In [9]:
len(jac_gdf)

57364

---

## Geography

#### LA County cities, unincorporated areas and LA City neighborhoods

In [10]:
hoods_gdf = gpd.read_file(
    "https://s3.us-west-1.amazonaws.com/stilesdata.com/la/la_city_hoods_county_munis.geojson"
).to_crs("EPSG:3857")

#### Clean up

In [11]:
hoods_gdf.columns = hoods_gdf.columns.str.lower()
hoods_gdf["coordinates"] = hoods_gdf.geometry.centroid
hoods_gdf["region_desc"] = (
    hoods_gdf.region.str.replace("-", " ").str.title().str.replace(" La", "")
)

#### Define the mapping from old type values to new descriptive values

In [12]:
type_mapping = {
    "standalone-city": "standalone city",
    "segment-of-a-city": "neighborhood in Los Angeles",
    "unincorporated-area": "unincorporated place in Los Angeles County",
}

#### Apply the mapping to the 'type_desc' column

In [13]:
hoods_gdf["type_desc"] = hoods_gdf["type"].map(type_mapping)

---

#### Merge hoods with trees

In [14]:
lahoods_merge = (gpd.sjoin(jac_gdf, hoods_gdf, predicate="within")).reset_index(
    drop=True
)[["id", "name", "type_desc", "city", "region_desc", "species", "geometry"]]

In [15]:
lahoods_merge["species"] = lahoods_merge["species"].str.lower()

In [16]:
lahoods_merge = lahoods_merge.to_crs(epsg=4326)
lahoods_merge["lat"] = lahoods_merge["geometry"].y
lahoods_merge["lon"] = lahoods_merge["geometry"].x

In [17]:
### Add a street view URL because Mapbox rejects lat/lon columns

In [18]:
lahoods_merge["street_view_url"] = lahoods_merge.apply(
    lambda row: f"http://maps.google.com/maps?q=&layer=c&cbll={row.lat},{row.lon}",
    axis=1,
)

#### Count how many jacarandas are in each place

In [19]:
lahoods_counts = (
    lahoods_merge.groupby("name")["geometry"]
    .count()
    .reset_index(name="jacaranda_count")
)

#### Merge that back to the geo file

In [ ]:
jacs_hoods = lahoods_merge.merge(lahoods_counts, on="name")[
    ["name", "type_desc", "region_desc", "jacaranda_count", "geometry"]
].reset_index(drop=True)

In [ ]:
jacs_hoods = gpd.GeoDataFrame(jacs_hoods).to_crs(epsg=4326).fillna("")

---

## Exports

#### GeoJSON

In [43]:
jacs_hoods.to_file(
    "../data/processed/lacounty_jacaranda_locations.geojson",
    driver="GeoJSON",
)

In [22]:
gdf.to_file(
    "../data/processed/la_county_tree_locations.geojson",
    driver="GeoJSON",
)

#### Upload to Mapbox

In [44]:
# Replace with your Mapbox access token
access_token = mapbox_key
params = {"access_token": access_token}

# Request S3 credentials to stage file
r = requests.post("https://api.mapbox.com/uploads/v1/stiles/credentials", params=params)

try:
    r.raise_for_status()
except requests.exceptions.HTTPError as err:
    print(f"Error uploading the GeoJSON file.")
    raise

creds = r.json()

# Path to your GeoJSON file
geojson_file_path = "../data/processed/lacounty_jacaranda_locations.geojson"

# Remove the 'crs' member from the GeoJSON
with open(geojson_file_path, "r") as f:
    geojson_data = json.load(f)

if "crs" in geojson_data:
    del geojson_data["crs"]

with open(geojson_file_path, "w") as f:
    json.dump(geojson_data, f)

# Upload file to S3
with open(geojson_file_path, "rb") as f:
    s3_client = boto3.client(
        "s3",
        aws_access_key_id=creds["accessKeyId"],
        aws_secret_access_key=creds["secretAccessKey"],
        aws_session_token=creds["sessionToken"],
    )
    s3_client.upload_fileobj(f, creds["bucket"], creds["key"])
    print(
        f"Uploaded {geojson_file_path} to S3 bucket {creds['bucket']} with key {creds['key']}"
    )

# Generate Tileset
headers = {"Cache-Control": "no-cache"}
payload = {
    "url": creds["url"],
    "tileset": "stiles.jacaranda_tree_locations_v4",  # Use a new unique ID for the new tileset
    "name": "jacaranda_tree_locations",
}
s = requests.post(
    "https://api.mapbox.com/uploads/v1/stiles",
    params=params,
    headers=headers,
    json=payload,
)

try:
    s.raise_for_status()
except requests.exceptions.HTTPError as err:
    print(f"Error generating tileset.")
    raise

print("Tileset generation request sent successfully.")
print(s.json())

Uploaded ../data/processed/lacounty_jacaranda_locations.geojson to S3 bucket tilestream-tilesets-production with key d0/_pending/bwbodxizkqp1z5n08dkl9lbmc/stiles
Tileset generation request sent successfully.
{'id': 'cmbl9lvuo3y161mn5dr54irgz', 'name': 'jacaranda_tree_locations', 'complete': False, 'error': None, 'created': '2025-06-06T20:35:43.177Z', 'modified': '2025-06-06T20:35:43.177Z', 'tileset': 'stiles.jacaranda_tree_locations_v4', 'owner': 'stiles', 'progress': 0}
